In [1]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

def getdocs():
  r = requests.get('https://www.kompas.com/')
  soup = BeautifulSoup(r.content, 'html.parser')

  link = []
  for i in soup.find('div', {'class': 'most__wrap'}).find_all('a'):
      i['href'] = i['href'] + '?page=all'
      link.append(i['href'])

# Getting the paragraphs
  doc = []
  for i in link:
      r = requests.get(i)
      soup = BeautifulSoup(r.content, 'html.parser')

      sen = []
      for i in soup.find('div', {'class': 'read__content'}).find_all('p'):
          sen.append(i.text)
      doc.append(' '.join(sen))

  # Cleaning up paragraphs
  doc_clean = []
  for d in doc:
      doc_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
      doc_test = re.sub(r'@\w+', '', doc_test)
      doc_test = doc_test.lower()
      doc_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', doc_test)
      doc_test = re.sub(r'[0-9]', '', doc_test)
      doc_test = re.sub(r'\s{2,}', ' ', doc_test)
      doc_clean.append(doc_test)

  return doc_clean

In [2]:
docs = getdocs()

# TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Creating dataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names())

In [3]:
def getArticles(q, df):
  print("Search term:", q)
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similar Article (Cosine sim. Value): ", v)
      print(docs[k])
      print()

terms = 'covid-19'

getArticles(terms, df)

Search term: covid-19
Similar Article (Cosine sim. Value):  0.04706576099036292
kompas com bupati bogor ade munawaroh yasin atau ade yasin ditangkap komisi pemberantasan korupsi kpk dalam operasi tangkap tangan ott operasi tangkap tangan dilakukan sejak selasa malam hingga rabu pagi selain ade yasin kpk juga mengamankan beberapa pihak dari badan pemeriksa keuangan bpk perwakilan jawa barat serta menyita sejumlah uang ironisnya dua hari sebelum ditangkap kpk ade yasin sempat mengeluarkan surat edaran tentang pencegahan korupsi dan pengendalian gratifikasi terkait hari raya baca juga ott bupati bogor ade yasin wabup saya takut salah kalau komentar lewat se tersebut ade melarang pejabat aparatur sipil negara asn pimpinan dan karyawan bumd meminta memberi dan menerima gratifikasi yang berhubungan dengan jabatan atau kewenangannya yang dikaitkan dengan hari raya atau pandemi covid wajib menjadi teladan yang baik bagi masyarakat dengan tidak melakukan permintaan pemberian dan penerimaan grat